In [2]:
#script for praseing the data from EPT excel file
# "Network information report"

import pandas as pd
import numpy as np
import os
import re
import openpyxl
import xlrd
import xlsxwriter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.styles import colors
from openpyxl.styles import Font, Color


#importing the data from excel file
df = pd.read_excel(r"D:\Autmation\My scripts\span Losses details form EPT\NetworkInformation_Report_TE_PO74_PO81_DM_REL11.1.2_27082022_ED4_8617777894366979498.xls", sheet_name='Fiber')

#drop last row & frist 5 rows
df = df.drop(df.index[0:5])
df = df.drop(df.index[-1])

#drop last 30 columns & frisrt 3 columns plus coulmns 5,6,7,8,10
df = df.drop(df.columns[[0,1,2,5,6,7,8,10,12,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,
                            -18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]], axis=1)

# Rename the columns
df = df.rename(columns={'Unnamed: 3':'FROM','Unnamed: 4':'TO', 'Unnamed: 9':'Dir'
                                        ,'Unnamed: 11':'km','Unnamed: 13':'Max','Unnamed: 14':'Min'})


#fill the empty cells with the value of the cell above it
df = df.fillna(method='ffill')

# to replace the value of the cell in the column 'Dir' with '<-->' with 'Bi'
df['Dir'] = df['Dir'].replace(to_replace=r'<-->', value='Bi', regex=True)

# to replace the value of the cell in the column 'Dir' with '-->' with 'Uni'
df['Dir'] = df['Dir'].replace(to_replace=r'-->', value='Uni', regex=True)

# to replace the value of the cell in the column 'Dir' with '<--' with 'reverse'
df['Dir'] = df['Dir'].replace(to_replace=r'<--', value='Rev', regex=True)

# if the value of the cell in the column 'Dir' is ' Rev' then swap the cell value in the column 'FROM' with the value of the cell in the columen 'TO' in same row
df['FROM'] = np.where(df['Dir'] == 'Rev', df['TO'], df['FROM'])

#if the value of the cell in the column 'Dir' is 'Bi' then insert a new row rplicating the current raw data
for i in range(5,len(df)):
    if df.loc[i].at['Dir'] != 'Rev ' and df.loc[i].at['Dir'] != '  Uni':
        df = df.append(df.loc[i], ignore_index=True)
  

# asending the values in the dataframe by the column 'FROM' only

df = df.sort_values(by=['FROM', 'TO'],ascending=False)

# reset the index of the dataframe
df = df.reset_index(drop=True)

# starting from second raw and each two raws swap the value of the cell in the column 'FROM' with the value of the cell in the columen 'TO' in same row
# and swap the value of the cell in the column 'TO' with the value of the cell in the columen 'FROM' in same row
for i in range(1,len(df),2):
    df.loc[i,'FROM'], df.loc[i,'TO'] = df.loc[i,'TO'], df.loc[i,'FROM']

#drop the column 'Dir'
df = df.drop(['Dir'], axis=1)

print(df)

# Save the main dataframe to an excel file
df.to_excel(r'D:\Autmation\My scripts\span Losses details form EPT\EPT_output.xlsx',index=False)

                 FROM                TO    km    Max    Min
0             Zagazig             ZAG E   4.0   8.00   3.00
1               ZAG E           Zagazig   4.0   8.00   3.00
2             Zagazig  10TH-RAMADAN-MTX  63.0  25.20  20.20
3    10TH-RAMADAN-MTX           Zagazig  63.0  27.00  23.30
4             Zagazig      10 th Ramdan  61.0  24.21  19.21
..                ...               ...   ...    ...    ...
121            Ramsis            Almaza  17.7  18.51  13.51
122            Almaza             Banha  69.0  29.00  24.00
123             Banha            Almaza  69.0  31.30  26.30
124      10 th Ramdan  10TH-RAMADAN-MTX   1.5   9.00   4.00
125  10TH-RAMADAN-MTX      10 th Ramdan   1.5   9.00   4.00

[126 rows x 5 columns]


C:\Users\omagdysa\AppData\Local\Temp\ipykernel_10600\41069111.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df.loc[i], ignore_index=True)
